In [ ]:
#SPOTIPY - FOR EXTRACTING SONG TITLES VIA SPOTIFY API

In [ ]:
import tqdm

In [26]:
!pip install spotipy

In [29]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd

In [30]:
scope='playlist-read-private'

In [31]:
 spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='spotify client id', client_secret='spotify client secret id', redirect_uri='http://localhost:9000', scope=scope))

In [32]:
username=spotify.me()['id']
playlistID='78Facqd3sp0ffs99iq7IdJ'

In [33]:
TrackList=[]
res=spotify.user_playlist_tracks(username, playlistID)

In [35]:
playlists=spotify.current_user_playlists()

In [36]:
len(playlists)

7

In [39]:
totaltracks=0
countlist=[]
for i in playlists['items']:

        print(i['name'])
        
        print(i['tracks']['total'])
        
        countlist.append(i['tracks']['total'])
        totaltracks+=i['tracks']['total']

        
        print(' ')

print('total songs=',totaltracks)

This Is Daft Punk
43
 
Soundtracks
7
 
Forza Horizon 5 Pulse (Official)
22
 
Kishore + MAN
50
 
Tamil Pop/Hip/Rap
91
 
Non-Stop-Pop FM (GTAV)
41
 
BUILT DIFFERENT
60
 
Racing
13
 
1970's Rock
9
 
20syl • 36 (Beats & Types)
36
 
Tamil
50
 
Fav
179
 
total songs= 601


In [40]:
countlist

[43, 7, 22, 50, 91, 41, 60, 13, 9, 36, 50, 179]

In [41]:
track_info=[]

In [42]:
def trackname(result):
    for i, item in enumerate(result['items']):
        
        track = item['track']
        
        title=track['artists'][0]['name']+' '+track['name']
        
        #print(track['artists'][0]['name'], track['name'])
        
        track_info.append(title)

In [43]:
for playlist in playlists['items']:
        #if playlist['owner']['id'] == username:
            results = spotify.playlist(playlist['id'], fields="tracks,next")
            
            tracks = results['tracks']
            trackname(tracks)

            
            while tracks['next']:
                tracks = spotify.next(tracks)
                trackname(tracks)   

In [45]:
totaltracks=len(track_info)

In [46]:
def extractorP(playlistchoice):
    j=playlistchoice
    offset=0

    for j in countlist[playlistchoice:]:
    
        offset+=j
       
    startOfP=totaltracks-offset
    endOfP=totaltracks-(offset-countlist[playlistchoice])

    desiredP=track_info[startOfP:endOfP]
    
    return desiredP
    

In [47]:
TRACKS=extractorP(11)
TRACKS

In [ ]:
#EXTRACT URL OF TRACKS THROUGH YOUTUBE API with query:

In [17]:
! pip install google-api-python-client

  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.53.0
    Uninstalling googleapis-common-protos-1.53.0:
      Successfully uninstalled googleapis-common-protos-1.53.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.25.1
    Uninstalling google-api-core-1.25.1:
      Successfully uninstalled google-api-core-1.25.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0, but you have google-api-core 2.8.2 which is incompatible.


In [68]:
import googleapiclient.discovery

api_name='youtube'
api_version='v3'

In [52]:
def switcher(api_key):
    yt=googleapiclient.discovery.build( api_name, api_version, developerKey=api_key)

In [ ]:
    
URLlist=[]    
count=0    
apikeyList=['xxxxxxxxxxxxxx','yyyyyyyyyyyyyyyy'] # needs 12 api keys to fetch 1200 urls at max per day
apiIndex=0
yt=googleapiclient.discovery.build( api_name, api_version, developerKey=apikeyList[apiIndex])


limit=[99, 198, 296] # current limit is 300. With 12 API keys, limit will be 1200 

for Tindex in tqdm(range(len(TRACKS))):    
    
    if len(limit)!=0:
        if Tindex==limit[0]:
            
            if len(limit)==1:
                print("The number of tracks has exceeded the quota. Process ended at song number: ",Tindex)
                break
            else:
                
                print("Switching api",Tindex)
                apiIndex=apiIndex+1
            
                api_key=apikeyList[apiIndex]
                yt=googleapiclient.discovery.build( api_name, api_version, developerKey=api_key)
                
        
            
                del limit[0]
                

    request=yt.search().list(
    
        part='id',
        type='video',
        q=TRACKS[Tindex],
        maxResults=1,
        fields='items(id(videoId))'
        )

    response=request.execute()

    vidID=response['items'][0]['id']['videoId']

    URL="https://www.youtube.com/watch?v="+vidID
    
    URLlist.append(URL)
    
    count+=1
    #print(count)

In [70]:
len(URLlist)

179

In [71]:
URLdf=pd.DataFrame(URLlist)

In [72]:
URLdf.to_csv("url.csv")

In [ ]:
#PYTUBE - DOWNLOADING YT VIDEOS

In [74]:
!pip install pytube

In [3]:
#URLdf=pd.read_csv('url.csv')

In [4]:
#URLlist=URLdf.values.tolist()

In [ ]:
from pytube import YouTube

count=0

for url in URLlist:
    youtube=YouTube(url)
    
    streams=youtube.streams.filter(only_audio=True).all()

    stream = youtube.streams.get_by_itag(251)
    stream.download()
    count+=1
    print(count)

In [75]:
count

179